<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="https://www.uoc.edu/content/dam/news/images/noticies/2016/202-nova-marca-uoc.jpg" align="left" width="45%">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;">M2.891 · Aprendizaje automático · PEC3</p>
<p style="margin: 0; text-align:right;">2024-1 · Máster universitario en Ciencia de datos (Data science)</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Estudios de Informática, Multimedia y Telecomunicación</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# PEC 3: Métodos supervisados

En esta práctica veremos diferentes métodos supervisados y trataremos de optimizar diferentes métricas. Veremos como los diferentes modelos clasifican las observaciones y con cuales obtenemos mayor rendimiento. Después aplicaremos todo lo que hemos aprendido hasta ahora a un dataset nuevo simulando un caso práctico real.

1. [Exploración de algoritmos supervisados](#eje1) \
    1.1. [Carga de datos](#eje10) \
    1.2. [Naive-Bayes](#eje11) \
    1.3. [Análisis Discriminante Lineal (LDA) y Análisis Discriminante Cuadrtático (QDA)](#eje12) \
    1.4. [K vecinos más próximos (KNN)](#eje13) \
    1.5. [Máquinas de soporte vectorial (SVM)](#eje14) \
    1.6. [Árboles de decisión](#eje15) 
2. [Implementación del caso práctico](#eje2)\
    2.1. [Carga de datos](#eje20) \
    2.2. [Análisis Exploratorio de Datos](#eje21) \
    2.3. [Preprocesamiento de Datos](#eje22) \
    2.4. [Modelización](#eje23) 


<u>Consideraciones generales</u>:

- La solución planteada no puede utilizar métodos, funciones o parámetros declarados **_deprecated_** en futuras versiones, a excepción de la carga de datos cómo se indica posteriormente.
- Esta PEC debe realizarse de forma **estrictamente individual**. Cualquier indicio de copia será penalizado con un suspenso (D) para todas las partes implicadas y la posible evaluación negativa de la asignatura de forma íntegra.
- Es necesario que el estudiante indique **todas las fuentes** que ha utilizado para la realización de la PEC. De no ser así, se considerará que el estudiante ha cometido plagio, siendo penalizado con un suspenso (D) y la posible evaluación negativa de la asignatura de forma íntegra.

<u>Formato de la entrega</u>:

- Algunos ejercicios pueden suponer varios minutos de ejecución, por lo que la entrega debe hacerse en **formato notebook** y en **formato html**, donde se vea el código, los resultados y comentarios de cada ejercicio. Se puede exportar el notebook a HTML desde el menú File $\to$ Download as $\to$ HTML.
- Existe un tipo de celda especial para albergar texto. Este tipo de celda os será muy útil para responder a las diferentes preguntas teóricas planteadas a lo largo de la actividad. Para cambiar el tipo de celda a este tipo, en el menú: Cell $\to$ Cell Type $\to$ Markdown.

<div class="alert alert-block alert-info">
    <strong>Nombre y apellidos:</strong>
</div>

In [ ]:
import os
import shap
import copy
import tqdm
import torch
import pickle
import kagglehub
import umap

import seaborn as sns
import pandas as pd
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
import torch.optim as optim

from matplotlib.colors import ListedColormap

from sklearn import tree
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, train_test_split
from torcheval.metrics.functional import binary_f1_score, binary_accuracy, binary_auroc

%matplotlib inline

<a id='ej1'></a>
# 1. Exploración de algoritmos supervisados

## 1.1. Carga de datos

El conjunto de datos Fashion MNIST proporcionado por Zalando consta de 70.000 imágenes con 10 clases diferentes de ropa repartidas uniformemente. No obstante, para esta práctica utilizaremos únicamente un subconjunto de 5.000 imágenes que consiste en 1.000 imágenes de 5 clases diferentes.

Las imágenes tienen una resolución de 28x28 píxeles en escala de grises, por lo que se pueden representar utilizando un vector de 784 posiciones.

El siguiente código cargará las 5.000 imágenes en la variable images y las correspondientes etiquetas (en forma numérica) en la variable labels. Podemos comprobar que la carga ha sido correcta obteniendo las dimensiones de estas dos variables.

In [ ]:
with open("data.pickle", "rb") as f:
    data = pickle.load(f)
    
X = data["images"]
y = data["labels"]
n_classes = 5
labels = ["T-shirt", "Trouser", "Pullover", "Dress", "Coat"]

print("Vector Image Dimensions: {}".format(X.shape))
print("Vector Label Dimensions: {}".format(y.shape))

Con el siguiente código podemos ver un ejemplo de imagen de cada una de las clases. Para ello reajustamos el vector de 784 dimensiones que representa cada imagen en una matriz de tamaño 28x28 y la transponemos para mostrarla:

In [ ]:
fig, ax = plt.subplots(1, n_classes, figsize=(10,10))

idxs = [np.where(y == i)[0] for i in range(n_classes)]

for i in range(n_classes):
    k = np.random.choice(idxs[i])
    ax[i].imshow(X[k].reshape(28, 28), cmap="gray")
    ax[i].set_title("{}".format(labels[i]))

plt.show()

<div class="alert alert-block alert-info">
    <strong>Implementación:</strong> 

Dividid el _dataset_ en dos subconjuntos, __*train*__ (80% de los datos) y __*test*__ (20% de los datos). Nombrad los conjuntos como: X_train, X_test, y_train, y_test. Utilizad la opción `random_state = 24`.
    
Podéis utilizar la implementación `train_test_split` de `sklearn`.
    
</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

Para poder visualizar los resultados de cada algoritmo supervisado, reduciremos el dataset anterior a dos dimensiones.

In [ ]:
model = umap.UMAP(n_components=2, random_state=42)
model.fit(X_train)
X_train_projection = model.transform(X_train)
X_test_projection = model.transform(X_test)

fig, ax = plt.subplots(1, 1, figsize=(10, 8))
for i in range(n_classes):
    ax.scatter(X_test_projection[y_test == i,0], X_test_projection[y_test == i,1], s=3, label=labels[i])
plt.legend()
plt.tight_layout()
plt.show()

A lo largo de los ejercicios aprenderemos a ver gráficamente las fronteras de decisión que nos devuelven los diferentes modelos. Para ello utilizaremos la función definida a continuación, que sigue los siguientes pasos:

Crear una meshgrid con los valores mínimo y máximo de 'x' e 'y'.
Predecir el clasificador con los valores de la meshgrid.
Hacer un reshape de los datos para tener el formato correspondiente.
Una vez hecho esto, ya podemos hacer el gráfico de las fronteras de decisión y añadir los puntos reales. Así veremos las áreas que el modelo considera que son de una clase y las que considera que son de otra. Al poner encima los puntos veremos si los clasifica correctamente en el área que les corresponde.

In [ ]:
# Create the meshgrid with the minimum and maximum values of the x and y axes
x_min, x_max = X_test_projection[:, 0].min() - 1, X_test_projection[:, 0].max() + 1
y_min, y_max = X_test_projection[:, 1].min() - 1, X_test_projection[:, 1].max() + 1

# Define the function that will visualize the decision boundary
def plot_decision_boundaries(model, X_test_projection, y_test):
    
    xx, yy = np.meshgrid(np.arange(x_min, x_max, 0.05),
                         np.arange(y_min, y_max, 0.05))
    
    # Prediction by using all values of the meshgrid
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])

    # Define the colors (one for each class)
    cmap_light = ListedColormap(['gainsboro','lightgreen','peachpuff','lightcyan', 'pink'])
    cmap_bold = ['grey','g','sandybrown','c','palevioletred']
    
    # Draw the borders
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(20,10))
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light)

    # Draw the points
    for i in range(n_classes):
        plt.scatter(X_test_projection[y_test == i,0], X_test_projection[y_test == i,1], 
                    s=3, label=labels[i], c=cmap_bold[i])
    plt.legend()
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.show()

<a id='ej11'></a>
## 1.2. Gaussian Naïve Bayes (1 punto)

El propósito de este primer ejercicio es comprender el funcionamiento del algoritmo Naïve-Bayes, un algoritmo peculiar que se basa en el teorema de Bayes para calcular la probabilidad de que una observación pertenezca a cada una de las clases. Este modelo asume que las características de entrada son independientes entre sí, lo que permite simplificar el cálculo de las probabilidades condicionales.

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>

1. **Entrena un Modelo de Naïve-Bayes:** Utiliza el conjunto de datos de _train_ para entrenar un modelo de Naïve-Bayes. Emplea el clasificador `GaussianNB` de la biblioteca `sklearn` para este fin.

2. **Calcula el _Accuracy_ del Modelo:** Una vez entrenado el modelo, calcula su precisión (_accuracy_) tanto en el conjunto de _train_ como en el de _test_. Esto te permitirá evaluar qué tan bien está funcionando tu modelo.

3. **Calcula la Matriz de Confusión:** Utiliza el conjunto de _test_ para calcular la matriz de confusión del modelo. Esta matriz te ayudará a entender de mejor manera los aciertos y errores de tu clasificador.

4. **Representa Gráficamente la Frontera de Decisión:** Finalmente, visualiza la frontera de decisión del modelo utilizando el conjunto de _test_. Puedes hacer esto con la ayuda de la función `plot_decision_boundary` que ya has creado previamente.

Para realizar estos cálculos y visualizaciones, utiliza las funciones `accuracy_score` y `confusion_matrix` del paquete `metrics` de `sklearn`.

</div>


<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Análisis:</strong> 
  
Análisis del ejercicio.

   - ¿Cómo son las fronteras de decisión? ¿Tiene sentido que tengan esta forma con el algoritmo utilizado?
   - ¿Cómo son las predicciones obtenidas sobre el conjunto de test?
</div>

<div class="alert alert-block alert-success">
<strong>Respuesta:</strong>


</div>

<a id='ej12'></a>
## 1.3 Análisis Discriminante Lineal (LDA) y Análisis Discriminante Cuadrático (QDA) (1 punto)

Ahora, analizarás dos algoritmos que se basan en la transformación lineal de las características de entrada para maximizar la separación entre las clases. Estos modelos operan bajo la suposición de que las características siguen una distribución gaussiana. Esto te permitirá calcular las probabilidades condicionales de cada clase. Con estos cálculos, asignarás a cada observación la clase que presente la mayor probabilidad condicional.

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>

Sigue estos pasos con el dataset de entrenamiento (_train_):    
    
1. Entrena un modelo de Análisis Discriminante Lineal (LDA) utilizando el clasificador `LinearDiscriminantAnalysis` de `sklearn`.
2. Calcula el _accuracy_ (precisión) del modelo tanto en los datos de _train_ como de _test_.
3. Calcula la matriz de confusión utilizando los datos de _test_.
4. Representa gráficamente la frontera de decisión con los datos de _test_.

Estas acciones te ayudarán a evaluar la eficacia del modelo LDA en tu conjunto de datos y a entender mejor cómo clasifica las observaciones.

</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Análisis:</strong>

1. Observa las fronteras de decisión que has generado. Reflexiona sobre su forma: ¿Se ajustan a lo que esperarías del algoritmo de Análisis Discriminante Lineal (LDA)? Considera la naturaleza lineal del algoritmo y cómo esto influye en la forma de las fronteras.
2. Evalúa las predicciones realizadas sobre el conjunto de test. Analiza su precisión y cómo se distribuyen respecto a las fronteras de decisión. ¿Son coherentes estas predicciones con lo que observas en las fronteras de decisión?

Estas reflexiones te permitirán comprender mejor la efectividad del modelo LDA y su adecuación para el conjunto de datos con el que estás trabajando."
</div>

<div class="alert alert-block alert-success">
<strong>Respuesta:</strong>


</div>

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>

Realiza los siguientes pasos con el dataset de entrenamiento (_train_):

1. Entrena un modelo de Análisis Discriminante Cuadrático (QDA) usando el clasificador `QuadraticDiscriminantAnalysis` de `sklearn`.
2. Calcula el _accuracy_ (precisión) del modelo tanto en los datos de _train_ como de _test_.
3. Calcula la matriz de confusión utilizando los datos de _test_.
4. Representa gráficamente la frontera de decisión con los datos de _test_.

Estos pasos te ayudarán a evaluar cómo el modelo QDA se comporta con tu conjunto de datos, y a entender su capacidad para clasificar y diferenciar entre las clases."

</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Análisis:</strong>

1. Examina las fronteras de decisión que has generado. Reflexiona sobre su forma: ¿Es coherente con lo que esperarías del algoritmo de Análisis Discriminante Cuadrático (QDA)? Considera cómo la naturaleza cuadrática del algoritmo podría influir en la forma de estas fronteras.
2. Evalúa las predicciones realizadas sobre el conjunto de test. Observa su precisión y cómo se distribuyen en relación con las fronteras de decisión. ¿Son estas predicciones consistentes con las fronteras observadas?
3. Reflexiona sobre las diferencias entre los algoritmos LDA y QDA. ¿En qué se distinguen en términos de supuestos, enfoque y resultados en tus datos?

Este análisis te permitirá comprender las características y la eficacia de ambos modelos, LDA y QDA, y cómo se aplican a tu conjunto de datos."
</div>

<div class="alert alert-block alert-success">
<strong>Respuesta:</strong>
</div>

<a id='ej13'></a>
## 1.4. KNN (1 punto)

En este punto, vas a entender el funcionamiento del algoritmo KNN (K-Nearest-Neighbor), que se basa en la proximidad de los puntos de datos en un espacio de características. Analizarás sus ventajas y desventajas, y comprenderás cómo los parámetros que lo componen influyen en su comportamiento.

KNN es un algoritmo de tipo supervisado basado en instancia. Esto significa:

- Supervisado: Tu conjunto de datos de entrenamiento está etiquetado con la clase o resultado esperado.
- Basado en instancia (_Lazy Learning_): El algoritmo no aprende explícitamente un modelo, como en la Regresión Logística o los árboles de decisión. En cambio, memoriza las instancias de entrenamiento y las utiliza como "conocimiento" en la fase de predicción.

Para entender cómo funciona KNN, sigue estos pasos:

1. Calcula la distancia entre el ítem a clasificar y los demás ítems del dataset de entrenamiento.
2. Selecciona los "k" elementos más cercanos, es decir, aquellos con la menor distancia, según el tipo de distancia que utilices (euclídea, coseno, manhattan, etc).
3. Realiza una "votación de mayoría" entre los k puntos seleccionados: la clase que predomine en estos puntos decidirá la clasificación final del ítem analizado.

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>

Realiza los siguientes pasos con el dataset de entrenamiento (_train_):

1. Entrena un clasificador KNN con el hiperparámetro `n_neighbors=2` usando el clasificador `KNeighborsClassifier` de `sklearn`.
2. Calcula el _accuracy_ (precisión) del modelo tanto en los datos de _train_ como de _test_.
3. Calcula la matriz de confusión utilizando los datos de _test_.
4. Representa gráficamente la frontera de decisión con los datos de _test_.

Si al entrenar el clasificador aparece un aviso (warning) y deseas ignorarlo, ejecuta el siguiente código antes del entrenamiento:

`import warnings`
`warnings.filterwarnings('ignore', message='^.*will change.*$', category=FutureWarning)`"

Esto te permitirá evaluar la efectividad del modelo KNN con `n_neighbors=2` en tu conjunto de datos, y entender cómo se comporta en términos de clasificación y separación de clases.    
    
    
    
</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

En el modelo que has entrenado, has fijado el parámetro `n_neighbors` de forma arbitraria. Sin embargo, es posible que con otro valor obtengas una mejor predicción. Para encontrar el valor óptimo de los parámetros de un modelo (_hyperparameter tunning_), a menudo se utiliza una búsqueda de rejilla (_grid search_). Esto implica entrenar un modelo para cada combinación posible de hiperparámetros y evaluarlo mediante validación cruzada (_cross validation_) con 5 particiones estratificadas. Luego, seleccionarás la combinación de hiperparámetros que haya obtenido los mejores resultados.

En este caso, te centrarás en optimizar un solo hiperparámetro:

- 𝑘: el número de vecinos que se consideran para clasificar un nuevo ejemplo. Debes probar con todos los valores entre 1 y 20.

Realiza este proceso para identificar el número óptimo de vecinos, lo que te permitirá mejorar la precisión de tus predicciones con el modelo KNN.

<div class="alert alert-block alert-info">
    <strong>Implementación:</strong>

Para calcular el valor óptimo del hiperparámetro _k_ (`n_neighbors`), debes realizar una búsqueda de rejilla con validación cruzada. Este proceso te ayudará a encontrar el valor óptimo de _k_. Para cada valor, calcula su promedio y la desviación estándar. Luego, implementa un _heatmap_ para visualizar la precisión según los diferentes valores del hiperparámetro.

Utiliza el módulo `GridSearchCV` de `sklearn` para calcular el mejor hiperparámetro. Para la visualización del _heatmap_, emplea la librería `Seaborn`.

Estos pasos te permitirán identificar de manera efectiva y visual el valor de _k_ que maximiza la precisión de tu modelo KNN."
</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>

Sigue estos pasos con el dataset de entrenamiento (_train_):

1. Entrena un clasificador KNN utilizando el mejor hiperparámetro que hayas encontrado.
2. Calcula el _accuracy_ (precisión) del modelo tanto en los datos de _train_ como de _test_.
3. Calcula la matriz de confusión utilizando los datos de _test_.
4. Representa gráficamente la frontera de decisión con los datos de _test_.

Este proceso te permitirá ver cómo el hiperparámetro óptimo que has identificado mejora la efectividad de tu modelo KNN en la clasificación de los datos.
</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Análisis:</strong>

1. Comenta los resultados obtenidos en la búsqueda del mejor hiperparámetro. Reflexiona sobre cómo varió el rendimiento del modelo con los diferentes valores de `n_neighbors`.
2. Analiza cómo se visualiza gráficamente el cambio del valor de `n_neighbors`. ¿Observas alguna tendencia o patrón claro? ¿Es coherente esta diferencia entre los dos gráficos al cambiar el parámetro?
3. Examina las fronteras de decisión que has generado. ¿La forma de estas fronteras tiene sentido dado el algoritmo KNN utilizado? Piensa en cómo la elección del número de vecinos influye en la forma de la frontera.
4. Evalúa las predicciones realizadas sobre el conjunto de test. Observa su precisión y cómo se distribuyen en relación con las fronteras de decisión. ¿Son estas predicciones consistentes con lo que observas en las fronteras de decisión?

Este análisis te ayudará a comprender la eficacia del modelo KNN con diferentes configuraciones de `n_neighbors` y su impacto en la clasificación de los datos."
</div>

<div class="alert alert-block alert-success">
<strong>Respuesta:</strong>


</div>

<a id='ej14'></a>
## 1.5. SVM (1 punto)

En esta sección, vas a explorar las Máquinas de Vectores de Soporte (SVM), que se basan en el concepto del _Maximal Margin Classifier_ y el hiperplano.

Un hiperplano en un espacio p-dimensional se define como un subespacio plano y afín de dimensiones p-1. En dos dimensiones, es una recta; en tres, un plano convencional. Para dimensiones mayores a tres, aunque no es intuitivo visualizarlo, el concepto se mantiene.

Cuando los casos son perfectamente separables de manera lineal, surgen infinitos posibles hiperplanos. Para seleccionar el clasificador óptimo, utiliza el concepto de _maximal margin hyperplane_, el hiperplano que se encuentra más alejado de todas las observaciones de entrenamiento. Este se define calculando la distancia perpendicular mínima (margen) de las observaciones a un hiperplano. El hiperplano óptimo es aquel que maximiza este margen.

En el proceso de optimización, debes tener en cuenta que solo las observaciones al margen o que lo violan (vectores soporte) influyen en el hiperplano. Estos vectores soporte son los que definen el clasificador.

#### Los _kernels_ en SVM

En situaciones donde no puedes encontrar un hiperplano que separe dos clases, es decir, cuando las clases no son linealmente separables, puedes utilizar el truco del núcleo (_kernel trick_). Este método te permite trabajar en una dimensión nueva donde es posible encontrar un hiperplano para separar las clases.

Al igual que con el KNN, las SVM también dependen de varios hiperparámetros. En este caso, te enfocarás en optimizar dos hiperparámetros:

1. **C**: la regularización, que es el valor de penalización de los errores en la clasificación. Este valor indica el compromiso entre obtener el hiperplano con el margen más grande posible y clasificar correctamente el máximo número de ejemplos. Debes probar los siguientes valores: 0.01, 0.1, 1, 10, 50, 100 y 200.
   
2. **Gamma**: un coeficiente que multiplica la distancia entre dos puntos en el kernel radial. En términos simples, cuanto más pequeño sea gamma, más influencia tendrán dos puntos cercanos. Debes probar los valores: 0.001, 0.01, 0.1, 1 y 10.

Para validar el rendimiento del algoritmo con cada combinación de hiperparámetros, utiliza la validación cruzada (_cross-validation_) con 4 particiones estratificadas."

<div class="alert alert-block alert-info">
    <strong>Implementación:</strong>


1. Calcula el valor óptimo de los hiperparámetros _C_ y _gamma_ utilizando una búsqueda de rejilla con validación cruzada. Este proceso te ayudará a encontrar los valores óptimos.
2. Para cada combinación de valores, calcula su promedio y la desviación estándar.
3. Haz un _heatmap_ para visualizar la precisión según los diferentes valores de los hiperparámetros.

Utiliza el módulo `GridSearchCV` de `sklearn` para calcular los mejores hiperparámetros con el clasificador SVC (de `SVM` de `sklearn`). Para la visualización del _heatmap_, emplea la librería `Seaborn`.

Estos pasos te permitirán identificar de manera efectiva y visual los valores de _C_ y _gamma_ que maximizan la precisión de tu modelo SVM.
</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>

Realiza los siguientes pasos con el dataset de entrenamiento (_train_):

1. Entrena un modelo SVM utilizando la mejor combinación de parámetros que hayas encontrado.
2. Calcula el _accuracy_ (precisión) del modelo tanto en los datos de _train_ como de _test_.
3. Calcula la matriz de confusión utilizando los datos de _test_.
4. Representa gráficamente la frontera de decisión con los datos de _test_.

Este proceso te permitirá ver cómo la mejor combinación de parámetros mejora la efectividad de tu modelo SVM en la clasificación de los datos.
</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Análisis:</strong>

1. Comenta los resultados obtenidos en la búsqueda de los mejores hiperparámetros. Reflexiona sobre cómo varió el rendimiento del modelo SVM con los diferentes valores de _C_ y _gamma_. Considera si los valores óptimos encontrados tienen sentido en el contexto de tu conjunto de datos.
2. Examina las fronteras de decisión que has generado con el modelo SVM. ¿La forma de estas fronteras es coherente con lo que esperarías del algoritmo utilizado? Piensa en cómo la combinación de hiperparámetros seleccionados podría influir en la forma de las fronteras.
3. Evalúa las predicciones realizadas sobre el conjunto de test. Observa su precisión y cómo se distribuyen en relación con las fronteras de decisión. ¿Son estas predicciones consistentes con lo que observas en las fronteras de decisión?

Este análisis te ayudará a comprender la eficacia del modelo SVM con los hiperparámetros seleccionados y su impacto en la clasificación de los datos."
</div>

<div class="alert alert-block alert-success">
<strong>Respuesta:</strong>


</div>

<a id='ej15'></a>
## 1.6. Árboles de decisión (1 punto)

En esta sección, vas a explorar los árboles de decisión, modelos predictivos que se basan en reglas binarias (si/no) para clasificar las observaciones según sus atributos y predecir el valor de la variable respuesta. Estos árboles pueden ser clasificadores, como en tu ejemplo, o regresores para predecir variables continuas.

#### Construcción de un Árbol

Para construir un árbol, sigue el algoritmo de *recursive binary splitting*:

1. Comienza en la parte superior del árbol, donde todas las observaciones pertenecen a la misma región.
2. Identifica todos los posibles puntos de corte para cada uno de los predictores. Estos puntos de corte son los diferentes niveles de los predictores.
3. Evalúa las posibles divisiones para cada predictor utilizando una medida específica. En los clasificadores, estas medidas pueden ser el *classification error rate*, el índice Gini, la entropía o el chi-square.

Comprender estos pasos te ayudará a entender cómo los árboles de decisión crean divisiones binarias para clasificar los datos y cómo estos pueden aplicarse tanto para clasificación como para regresión.

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>

Sigue estos pasos:

1. Con el dataset de entrenamiento (_train_), entrena un árbol de decisión utilizando el clasificador `DecisionTreeClassifier` de la biblioteca `tree` de `sklearn`.
2. Calcula el _accuracy_ (precisión) del modelo tanto en los datos de _train_ como de _test_.
3. Calcula la matriz de confusión utilizando los datos de _test_.
4. Representa gráficamente la frontera de decisión con los datos de _test_.
5. Representa el árbol de decisión. Puedes utilizar el comando `plot.tree` de la biblioteca `tree` de `sklearn`.

Estos pasos te permitirán evaluar cómo el árbol de decisión se comporta en tu conjunto de datos, tanto en términos de clasificación como en su representación visual."
</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Análisis:</strong>

1. Evalúa y comenta los resultados obtenidos con el árbol de decisión. Considera tanto el _accuracy_ del modelo en los conjuntos de _train_ y _test_ como los resultados de la matriz de confusión.
2. Reflexiona sobre cómo la frontera de decisión visualizada en el conjunto de _test_ se alinea con los resultados obtenidos. ¿Es coherente con lo que esperarías de un árbol de decisión?
3. Observa la representación gráfica del árbol. Analiza cómo las diferentes ramificaciones y decisiones tomadas en el árbol explican el comportamiento del modelo y su impacto en la clasificación de los datos.

Este análisis te ayudará a comprender en profundidad el funcionamiento y la eficacia del árbol de decisión en tu conjunto de datos específico.

</div>

<div class="alert alert-block alert-success">
<strong>Respuesta:</strong>


</div>

#### Evitando el *overfitting*

El proceso de construcción de árboles descrito tiende a reducir rápidamente el error de entrenamiento, por lo que generalmente el modelo se ajusta muy bien a las observaciones utilizadas como entrenamiento (conjunto de *train*). Como consecuencia, los árboles de decisión tienden al *overfitting*.
   
Para evitar el *overfitting* en los árboles de decisión, es crucial que modifiques ciertos hiperparámetros del modelo de la siguiente manera:

1. Utiliza el hiperparámetro `max_depth`, que define la profundidad máxima del árbol. Deberás explorar los valores entre 4 y 10 para encontrar el equilibrio adecuado entre la complejidad del modelo y su capacidad para generalizar.
2. Establece el hiperparámetro `min_samples_split`, que es el número mínimo de observaciones que debe tener una hoja del árbol antes de considerar una división. Experimenta con valores como 2, 10, 20, 50 y 100 para asegurarte de que el árbol no se vuelva demasiado específico para las observaciones de entrenamiento.

Ajustando estos hiperparámetros, podrás controlar la tendencia del árbol de decisión a sobreajustarse al conjunto de entrenamiento, mejorando así su capacidad para realizar predicciones efectivas en nuevos datos."

<div class="alert alert-block alert-info">
    <strong>Implementación:</strong>

1. Calcula el valor óptimo de los hiperparámetros `max_depth` y `min_samples_split` utilizando una búsqueda de rejilla con validación cruzada. Este proceso te ayudará a encontrar los valores óptimos que evitarán el sobreajuste.
2. Para cada combinación de valores, calcula su promedio y la desviación estándar.
3. Haz un _heatmap_ para visualizar la precisión según los diferentes valores de los hiperparámetros.

Utiliza el módulo `GridSearchCV` de `sklearn` para calcular los mejores hiperparámetros con el clasificador `DecisionTreeClassifier` de `tree` de `sklearn`. Para la visualización del _heatmap_, emplea la librería `Seaborn`.

Estos pasos te permitirán identificar de manera efectiva y visual los valores de `max_depth` y `min_samples_split` que maximizan la precisión de tu árbol de decisión, minimizando el riesgo de sobreajuste.
</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>


1. Entrena un árbol de decisión con el dataset de entrenamiento (_train_) utilizando la mejor combinación de parámetros que hayas encontrado.
2. Calcula el _accuracy_ (precisión) del modelo tanto en los datos de _train_ como de _test_.
3. Calcula la matriz de confusión utilizando los datos de _test_.
4. Representa gráficamente la frontera de decisión con los datos de _test_.
5. Representa el árbol de decisión.

Estos pasos te permitirán evaluar cómo el árbol de decisión, ajustado con los hiperparámetros óptimos, se comporta en tu conjunto de datos, tanto en términos de clasificación como en su representación visual."
    
</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Análisis:</strong>

1. Evalúa y comenta los resultados obtenidos en la búsqueda de los mejores hiperparámetros. Considera cómo la combinación óptima de `max_depth` y `min_samples_split` ha impactado el rendimiento del árbol de decisión.
2. Examina las fronteras de decisión generadas con el conjunto de _test_. Reflexiona sobre si la forma de estas fronteras es coherente con lo que esperarías de un árbol de decisión configurado con estos hiperparámetros.
3. Analiza las predicciones realizadas sobre el conjunto de test. Observa su precisión y cómo se distribuyen en relación con las fronteras de decisión. ¿Son consistentes estas predicciones con la estructura del árbol de decisión y las fronteras observadas?

Este análisis te ayudará a comprender la eficacia del árbol de decisión con los hiperparámetros seleccionados y su impacto en la clasificación de los datos.
</div>

<div class="alert alert-block alert-success">
<strong>Respuesta:</strong>


</div>

<a id='eje2'></a>
# 2. Implementación del caso práctico (5 puntos)

Hoy en día, la logística de la última milla es un problema abordado en la industria por muchas empresas dedicadas al comercio electrónico. La información proporcionada al usuario a la hora de realizar un pedido puede suponer un valor diferencial. Por ello, muchas empresas dedican muchos recursos para dar una estimación precisa sobre el tiempo que va a tardar en llegar cada pedido. En este ejercicio nos vamos a centrar en predecir el nivel de servicio de las operaciones logísticas de última milla de Amazon. En concreto identificaremos aquellas entregas que se consideren premium (tiempo de reparto inferior a dos horas).

Para ello, vamos a utilizar el conjunto de datos de entregas [amazon-delivery-dataset](https://www.kaggle.com/datasets/sujalsuthar/amazon-delivery-dataset), el cual incluye datos sobre más de 43.632 entregas en varias ciudades, con información relevante sobre los detalles del pedido, los agentes de entrega, las condiciones meteorológicas y del tráfico, y las métricas de rendimiento de la entrega. En concreto, el dataset contiene 16 características:

- Order_ID: identificador único de pedido
- Agent_Age: edad del agente (repartidor)
- Agent_Rating: puntuación del agente (repartidor)
- Store_Latitude: latitud del almacén o tienda
- Store_Longitude: longitud del almacén o tienda
- Drop_Latitude: latitud del cliente
- Drop_Longitude: longitud del cliente
- Order_Date: fecha del pedido
- Order_Time: hora del pedido 
- Pickup_Time: hora a la que el pedido fue recogido para su entrega
- Weather: información sobre la climatología
- Traffic: información sobre el tráfico
- Vehicle: información sobre el vehículo
- Area: información sobre el área de reparto
- Category: categoría de los productos del pedido
- Delivery_Time: tiempo de reparto (minutos)

El objetivo de esta sección es abordar el análisis de este conjunto de datos y entrenar una red neuronal (Perceptrón Multicapa) para predecir el nivel de servicio. Aquí tienes algunos pasos que podrías seguir:

1. **Análisis Exploratorio de Datos (EDA)**: Comienza explorando el conjunto de datos para comprender su estructura y distribución. Analiza la proporción de cada clase. Observa la distribución de las diferentes características y su relación con la clase objetivo "class".

2. **Preprocesamiento de Datos**: Considera normalizar las características para que estén en la misma escala que las componentes principales.

3. **Modelización**: Utiliza un perceptrón multicapa como herramienta de clasificación. Dado que el objetivo es identificar el nivel de servicio de la entrega, es vital centrarse en métricas como la precisión, la sensibilidad (recall), el valor F1 y el área bajo la curva ROC (AUC-ROC).

4. **Evaluación**: Realiza una evaluación y análisis riguroso del rendimiento de tu modelo.

Este enfoque integral te permitirá no solo construir un modelo efectivo sino también comprender mejor las características subyacentes del nivel de servicio en el conjunto de datos.

<a id='ej20'></a>
## 2.1. Carga de datos y procesamiento inicial (0.5 puntos)

Lo primero que debes hacer es cargar el conjunto de datos y visualizar información relevante del mismo. Asegúrate de verificar lo siguiente:

1. Confirma la cantidad total de filas y columnas en el DataFrame.
2. Revisa el nombre de cada columna del DataFrame.
3. Verifica el número de valores no nulos en cada columna.
4. Identifica el tipo de datos de cada columna, que puede ser int, float, object, entre otros.
5. Comprueba la cantidad de memoria utilizada por el DataFrame.

Estos pasos te proporcionarán una comprensión inicial clara y detallada del conjunto de datos con el que estás trabajando.

In [ ]:
root_path = kagglehub.dataset_download("sujalsuthar/amazon-delivery-dataset")
dataset_path = os.path.join(root_path, "amazon_delivery.csv") 
data = pd.read_csv(dataset_path)

data.info()

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>

Cómo se puede observar, no disponemos del nivel de servicio en el conjunto de datos. Por ello, vamos a definir que todas las entregas realizadas en un máximo de dos horas han tenido un servicio Premium. Para ello, crea una nueva columna denominada "Premium_Delivery", que contenga el valor 1 si la entrega se ha realizado en un máximo de 120 minutos, y un 0 en caso contrario. Es importante asegurar que el tipo de la nueva columna sea entero.
</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>

Para simplificar el ejercicio y facilitar su comprensión, se deben eliminar las siguientes columnas: Order_ID, Order_Date, Order_Time y Pickup_Time
</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>
    
A continuación, vamos a calcular la distancia harvesiana en kilómetros entre el almacén y el cliente. Para ello, debemos crear un nueva columna "Distance" y eliminar las cuatro columnas relacionadas con las coordenadas.
</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<a id='ej21'></a>
## 2.2. Análisis Exploratorio de Datos (EDA) (1.25 puntos)

El Análisis Exploratorio de Datos (EDA, por sus siglas en inglés) en ciencia de datos es un enfoque inicial para comprender y resumir el contenido de un conjunto de datos. Este proceso implica varias técnicas y pasos:

1. **Inspección de Datos**: Se comienza por revisar los datos brutos para identificar su estructura, tamaño y tipo (como numérico, categórico). Esto incluye detectar valores faltantes o inusuales.

2. **Resumen Estadístico**: Se calculan estadísticas descriptivas como la media, mediana, rango, varianza y desviación estándar para obtener una idea general de las tendencias y patrones en los datos.

3. **Visualización de Datos**: Se utilizan gráficos y diagramas (como histogramas, gráficos de caja, diagramas de dispersión) para visualizar distribuciones, relaciones entre variables y posibles anomalías. Esto ayuda a comprender mejor los datos y a identificar patrones o irregularidades.

4. **Análisis de Relaciones y Correlaciones**: Se exploran las relaciones entre diferentes variables para entender cómo se influencian entre sí. Esto puede implicar el uso de matrices de correlación y gráficos de dispersión.

5. **Identificación de Patrones y Anomalías**: Se buscan patrones consistentes o anomalías (como valores atípicos) que puedan sugerir tendencias o problemas en los datos.

El EDA es una fase crítica en cualquier proyecto de ciencia de datos, ya que proporciona una comprensión profunda y una base sólida para posteriores análisis y modelado.

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>

1. Calcula las frecuencias de la variable objetivo (`Premium_Delivery`) en tu conjunto de datos. 
2. Crea un gráfico de barras para visualizar estas frecuencias. Esto te ayudará a entender la proporción de entregas premium en comparación con las que no lo son.

A continuación, analiza la distribución de las variables numéricas:

1. Representa gráficamente el histograma de las variables, separando las observaciones según la clase a la que pertenecen (premium o no).
2. Organiza todos los histogramas en un formato de 4 filas y 1 columna. Esto facilitará la comparación visual de las distribuciones para cada clase en cada variable.

Por último, analiza la distribución de las variables categóricas de forma análoga a las variables numéricas, organizando todos los histogramas en un formato de 5 filas y 1 columna.

Estos pasos te permitirán obtener una visión más clara de la estructura de tu conjunto de datos y cómo las diferentes variables pueden influir en la identificación de las entregas premium.
    </div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Análisis:</strong>

1. Evalúa la relación de las frecuencias de la variable `Premium_Delivery`. Reflexiona sobre cómo se distribuyen las transacciones entre premium y no no premium. ¿Es la distribución significativamente desigual? ¿Qué implica esto para el análisis y la modelización de los datos?
2. Analiza la información proporcionada por los histogramas de las variables descriptoras. Observa si hay diferencias notables en las distribuciones de estas variables entre las clases. Pregúntate: ¿Hay variables que muestren patrones distintos para el nivel de servicio?
3. Considera si hay otras formas de visualización que podrían ser útiles para entender mejor los datos. Por ejemplo, ¿serían útiles los diagramas de caja (boxplots) para visualizar la distribución de las variables en ambas clases? ¿Podría un mapa de calor de la matriz de correlación entre variables ayudarte a entender las relaciones entre ellas?

Este análisis te ayudará a obtener una comprensión más profunda de la naturaleza de tus datos y a identificar posibles características que podrían ser importantes para detectar las entregas premium.
</div>

<div class="alert alert-block alert-success">
<strong>Respuesta:</strong>
</div>

<a id='ej22'></a>
## 2.3. Preprocesamiento de Datos (1.25 puntos)

El preprocesamiento de datos en ciencia de datos es un paso crucial que involucra la preparación y transformación de datos brutos en un formato adecuado para su posterior análisis y modelado. Este proceso incluye varias tareas esenciales:

1. **Limpieza de Datos**: Se eliminan o corrigen datos erróneos, incompletos, inexactos o irrelevantes. Esto puede incluir tratar con valores faltantes, corregir errores de entrada y manejar outliers.

2. **Normalización y Escalado**: Los datos se transforman para que estén en una escala común, sin distorsionar diferencias en los rangos de valores ni perder información. Por ejemplo, escalado min-max o estandarización.

3. **Codificación de Variables Categóricas**: Las variables categóricas (como género o país) se convierten en formatos numéricos para que puedan ser procesadas por algoritmos de aprendizaje automático, utilizando técnicas como codificación one-hot o codificación de etiquetas.

4. **División de Datos**: Los datos se dividen en conjuntos de entrenamiento, validación y prueba, permitiendo entrenar modelos, afinar hiperparámetros y evaluar el rendimiento del modelo de manera efectiva.

5. **Manejo de Datos Desbalanceados**: En casos de conjuntos de datos desbalanceados, se aplican técnicas como sobremuestreo o submuestreo para asegurar que el modelo no esté sesgado hacia la clase más frecuente.

6. **Ingeniería de Características**: Se crean nuevas variables (características) a partir de los datos existentes para mejorar la capacidad del modelo para aprender patrones y hacer predicciones.

El preprocesamiento es esencial para mejorar la calidad de los datos y hacerlos más adecuados y efectivos para análisis y modelado en proyectos de ciencia de datos.

<div class="alert alert-block alert-info">
<strong>Implementación:</strong> elimina los atributos categóricos del conjunto de datos y en su lugar introduce la transformación de dichos atributos a tantas variables binarias como categorías tengan. Es importante que las nuevas columnas generadas sean de tipo entero. Recuerda que la codificación one-hot convierte las etiquetas categóricas en vectores binarios. En estos vectores, el valor de 1 se asigna a la posición correspondiente a la clase y el valor de 0 a todas las demás posiciones. Esto facilita que los modelos de aprendizaje automático procesen y entiendan las etiquetas categóricas.
<hr>
Sugerencia: utilizad la función "get_dummies" de "pandas".

</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>

En la primera sección del ejercicio pudimos observar que la columna Agent_Rating tenía un número reducido de valores nulos. En este ejercicio tenemos que imputar los valores nulos por el mínimo valor de la columna y verificar que ninguna columna adicional tiene valores nulos.
</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>

Ahora vamos a realizar la división del conjunto de datos, para ello sigue estos pasos:

1. Separa los descriptores de la variable respuesta. Asigna los descriptores al conjunto `X` y la variable respuesta al conjunto `y`.
2. Elimina del conjunto de descriptores la columna `Delivery_Time`, dado que fue la que utilizamos para calcular nuestra variable respuesta.
3. Divide el _dataset_ en dos subconjuntos: uno para entrenamiento (_train_) y otro para pruebas (_test_). Asigna el 80% de los datos al conjunto de entrenamiento (`X_train`, `y_train`) y el 20% al conjunto de pruebas (`X_test`, `y_test`). Utiliza la función `train_test_split` de la biblioteca `model_selection` de `sklearn`. Asegúrate de usar `random_state = 24` y haz una división estratificada para mantener la misma proporción de clases en ambos conjuntos.

</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>

1. Normaliza los descriptores utilizando el `StandardScaler` de `sklearn`. Esto estandarizará las características restando la media y dividiendo por la desviación estándar.
2. Muestra las dimensiones del conjunto de descriptores original, del conjunto de entrenamiento y del conjunto de prueba. Esto te permitirá ver cómo se han dividido los datos.

<strong>Nota:</strong> Ajusta el `StandardScaler` únicamente con los descriptores de entrenamiento para evitar la fuga de información o 'data leakage'. La fuga de información ocurre cuando se utiliza información del conjunto de prueba o validación en el proceso de ajuste del modelo. Es decir, si ajustas el modelo de escalado con todo el conjunto de datos, estarías utilizando información del conjunto de prueba o validación en el ajuste, lo que podría dar la impresión de que el modelo es más preciso de lo que realmente es. Por lo tanto, asegúrate de ajustar el `StandardScaler` solo con los datos de entrenamiento y luego aplicarlo a los conjuntos de entrenamiento y prueba.
</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>

Convierte los conjunto de datos de entrenamiento y test en tensores, utilizando el método `tensor` de la librería `PyTorch`.
</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<a id='ej23'></a>
## 2.4. Modelización (2 puntos)

El MLP (Perceptrón Multicapa) es, sin duda, una poderosa herramienta en el campo del aprendizaje automático y la inteligencia artificial. Puede manejar tareas de clasificación y regresión, lo que lo hace versátil para una variedad de problemas. Su capacidad para modelar relaciones no lineales complejas lo convierte en una elección popular cuando los datos no siguen patrones lineales simples.

Aquí hay algunos puntos clave sobre el MLP:

- **Capas y Neuronas**: El MLP consta de múltiples capas de neuronas, que incluyen una capa de entrada, una o más capas ocultas y una capa de salida. Cada neurona en una capa está conectada a todas las neuronas en la capa siguiente.

- **Funciones de Activación**: Para introducir no linealidad en el modelo, se utilizan funciones de activación en las neuronas, como la función sigmoide, ReLU (Rectified Linear Unit) o tangente hiperbólica. Estas funciones permiten al MLP capturar patrones complejos en los datos.

- **Aprendizaje Supervisado**: El entrenamiento del MLP implica ajustar los pesos de las conexiones entre neuronas para minimizar la diferencia entre las salidas producidas por la red y las salidas deseadas. Esto se hace utilizando algoritmos de aprendizaje supervisado, como el descenso del gradiente.

- **Ajuste de Hiperparámetros**: Al igual que otros modelos de aprendizaje automático, el MLP tiene hiperparámetros importantes, como el número de capas ocultas, el número de neuronas en cada capa, la función de activación y la tasa de aprendizaje. A menudo, es necesario ajustar estos hiperparámetros para obtener un buen rendimiento en una tarea específica.

- **Generalización**: Uno de los desafíos en el entrenamiento de MLP es evitar el sobreajuste (overfitting), donde el modelo se adapta demasiado a los datos de entrenamiento y no generaliza bien a datos nuevos. La regularización y la validación cruzada son técnicas comunes para abordar este problema.

En este contexto, el MLP puede ser una excelente opción para modelar patrones complejos que indiquen cuando una entrega será premium. Sin embargo, es importante ajustar y evaluar cuidadosamente el modelo para garantizar que funcione de manera efectiva en esta tarea crítica.

Crear y entrenar un MLP con varias capas ocultas con función de activación ReLU es una excelente elección. La función de activación ReLU (Rectified Linear Unit) es comúnmente utilizada en capas ocultas de redes neuronales debido a su capacidad para introducir no linealidad en el modelo, lo que le permite aprender patrones complejos en los datos.

Por otra parte, el enfoque de apilar capas lineales utilizando la clase `Linear` de PyTorch es una forma eficaz y sencilla de construir modelos de redes neuronales.

In [ ]:
def train_model(model, X_train, y_train, X_val, y_val, n_epochs, batch_size):
    batch_start = torch.arange(0, len(X_train), batch_size)
    optimizer = optim.Adam(model.parameters(), lr=0.0001)
    loss_fn = nn.BCELoss()

    best_acc = - np.inf
    best_weights = None

    train_loss_hist = []
    train_acc_hist = []
    val_loss_hist = []
    val_acc_hist = []
 
    for epoch in range(n_epochs):
        epoch_loss = []
        epoch_acc = []
        model.train()
        
        with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
            bar.set_description(f"Epoch {epoch}")
            for start in bar:

                # take a batch
                X_batch = X_train[start:start+batch_size]
                y_batch = y_train[start:start+batch_size]

                # forward pass
                y_pred = model(X_batch)
                loss = loss_fn(y_pred, y_batch)

                # backward pass
                optimizer.zero_grad()
                loss.backward()

                # update weights
                optimizer.step()

                # compute and store metrics
                acc = (y_pred.round() == y_batch).float().mean()
                epoch_loss.append(float(loss))
                epoch_acc.append(float(acc))
                bar.set_postfix(
                    loss=float(loss),
                    acc=float(acc)
                )

        # Evaluating the model at the end of each epoch
        model.eval()
        y_pred = model(X_val)
        ce = float(loss_fn(y_pred, y_val))
        acc = float((y_pred.round() == y_val).float().mean())

        train_loss_hist.append(np.mean(epoch_loss))
        train_acc_hist.append(np.mean(epoch_acc))
        val_loss_hist.append(ce)
        val_acc_hist.append(acc)

        if acc > best_acc:
            best_acc = acc
            best_weights = copy.deepcopy(model.state_dict())
        # print(f"Epoch {epoch} validation: Cross-entropy={ce:.2f}, Accuracy={acc*100:.1f}%")

    model.load_state_dict(best_weights)
    return best_acc, train_loss_hist, val_loss_hist, train_acc_hist, val_acc_hist

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>
    
La modealización del MLP la vamos a realizar con la librería `PyTorch`. Para ello:

1. Comienza creando el modelo `BinaryServiceLevel`, para lo cual es necesario crear una clase que herede de `nn.Module`.
2. En el constructor (`__init__`), declara las siguientes capas:
    - Una capa lineal `nn.Linear` de entrada con un tamaño de salida de 19, y una función de activación ReLu `nn.ReLu`.
    - Una capa lineal `nn.Linear` con un tamaño de salida de 19, y una función de activación ReLu `nn.ReLu`.
    - Una capa lineal `nn.Linear` de salida con una función de activación Sigmoid `nn.Sigmoid`.
3. Después, en el método `forward` enlaza las diferentes capas y sus respectivas funciones de activación en el orden definido en el punto anterior. 
4. No olvides mostrar el número de parámetros utilizando el método `.parameters()` del modelo.
</div>

In [ ]:
class BinaryServiceLevelBase(nn.Module):
    def __init__(self):
        super().__init__()
 
    def forward(self, x):
        return x

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>
  
1. Ahora, es hora de entrenar y validar el modelo aplicando validación cruzada utilizando `StratifiedKFold` sobre el conjunto de entrenamiento, con un valor de k = 5 y shuffle = True.
2. En cada split, el modelo se debe entrenar utilizando la función `train_model`. Asegúrate de entrenar con los datos de entrenamiento y validación de cada split, establece el número de épocas en 15 y el tamaño del lote en 32.
3. En cada iteración se deben calcular las siguientes métricas:
    - Calcula la exactitud (accuracy) para medir la exactitud de las predicciones.
    - Calcula el valor F1, que es una medida que combina exactitud y sensibilidad.
    - Calcula el área bajo la curva ROC (AUC-ROC) para evaluar el rendimiento del modelo en la clasificación binaria.
4. Por último, se debe mostrar la media de cada de las métricas calculadas en el punto anterior.

</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Análisis:</strong>
    
1. Realiza un análisis de los resultados y decide si consideras que este modelo es aceptable.
2. Evalúa cuál de las medidas de rendimiento utilizadas es la más apropiada.
3. Examina la distribución de las clases y plantea una estrategia, si es necesario, para asegurar la confiabilidad del estudio realizado.
</div>

<div class="alert alert-block alert-success">
<strong>Respuesta:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>
  
Ahora, es hora de analizar la fase de entrenamiento, para ello:
 
1. Divide el dataset de entrenamiento en dos subconjuntos a su vez: uno para entrenamiento (train) y otro para validación (val), asignando el 80% de los datos al conjunto de entrenamiento.
2. Entrena el modelo con la función `model_train`, guardando todos los parámetros que devuelve.
3. Crea gráficos que muestren la pérdida (`loss`) tanto en el entrenamiento como en la validación a lo largo de las épocas.
4. Por último, genera gráficos que representen la exactitud (`accuracy`) en el entrenamiento y la validación a lo largo de las épocas.

  
</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Análisis:</strong>

¿Que conclusiones puedes obtener de las gráficas tanto de la pérdida (`loss`) como de la exactitud (`accuracy`) en el entrenamiento y la validación a lo largo de las épocas?
</div>

<div class="alert alert-block alert-success">
<strong>Respuesta:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>
  
Es hora evaluar el rendimiento del modelo en el conjunto de test. Para ello:
   
1. Realiza la predicción sobre el conjunto de test.
2. Calcula las métricas de los apartados anteriores: accuracy, f1 score y curva roc.
 
</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Implementación:</strong>

Para analizar el modelo entrenado, nos vamos a apoyar en los [Shapley values](https://en.wikipedia.org/wiki/Shapley_value), los cuales nos introducen a la explicación de modelos de aprendizaje automático. El objetivo es explicar la predicción de un modelo calculando la contribución de cada característica a la predicción. La explicación técnica del concepto de SHAP es el cálculo de los valores de Shapley a partir de la teoría de juegos. En pocas palabras, los valores de Shapley son un método para mostrar el impacto relativo de cada característica (o variable) que estamos midiendo en el resultado final del modelo de aprendizaje automático comparando el efecto relativo de las entradas con la media.

Para calcular los _shap values_:

1. Selecciona una muestra de 10000 registros del conjunto de entrenamiento.
2. Inicializa el 'explainer' `shap.DeepExplainer` con el modelo entrenado y la muestra anterior.
3. Selecciona una muestra de 400 registros del conjunto de entrenamiento.
4. Calcula los _shap_ values utilizando la muestra anterior
5. Define y muestra un DataFrame con tres columnas
   - Media aritmética del valor absoluto de los valores
   - Desviación típica del valor absoluto de los valores
   - Nombre del atributo descriptivo
6. Muestra la representación gráfica de los valores utilizando la librería shap.
 
</div>

<div class="alert alert-block alert-danger">
<strong>Solución:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Análisis:</strong>
    
Relaciona la interpretación de los _shap values_ con el análisis exploratorio de los datos realizado en el ejercicio 2.2.
</div>

<div class="alert alert-block alert-success">
<strong>Respuesta:</strong>
</div>

<div class="alert alert-block alert-info">
<strong>Análisis:</strong>
    
Imagina que calculamos los _shap values_ para cada split en el ejercicio en el que entrenamos el modelo utilizando validación cruzada. 
- ¿Los análisis de los diferentes modelos deberían ser similares? ¿Por qué o por qué no?
- ¿Qué indicaría si el análisis de cada modelo varía mucho de uno a otro?
- ¿Qué usos adicionales les podemos dar a los _shap values_?
</div>

<div class="alert alert-block alert-success">
<strong>Respuesta:</strong>
</div>